# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tual,-5.6667,132.7500,29.18,75,100,4.13,ID,1704632985
1,1,st. john's,47.5649,-52.7093,-8.35,80,26,8.94,CA,1704632986
2,2,jamestown,42.0970,-79.2353,-0.23,100,100,3.09,US,1704632986
3,3,puerto natales,-51.7236,-72.4875,10.25,66,75,8.23,CL,1704632987
4,4,farasan,16.7022,42.1183,26.07,74,35,2.48,SA,1704632987


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot

# Converting the DataFrame to a GeoDataFrame
geometry = [Point(xy) for xy in zip(city_data_df['Lng'], city_data_df['Lat'])]
gdf = gpd.GeoDataFrame(city_data_df, geometry=geometry)

# Plotting the GeoDataFrame using hvplot
map_plot = gdf.hvplot.points('Lng', 'Lat', size='Humidity', geo=True, tiles='OSM',
                             color='City', hover_cols=['City', 'Humidity'], scale = 0.9,
                             title='City Map with Humidity').opts(width=1000, height=600)

# Displaying the map 
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Narrowing down cities
criteria = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Applying the criteria to filter cities
null_values_cities_df = criteria.dropna()

# Dropping any rows with null values
# null_values_cities_df = null_values_cities_df.dropna()

# Displaying sample data
null_values_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,port alfred,-33.5906,26.8910,23.43,77,0,3.58,ZA,1704632996
39,39,mar del plata,-38.0023,-57.5575,24.97,77,0,2.24,AR,1704632994
44,44,jamuria,23.7000,87.0833,22.36,44,0,2.09,IN,1704633011
55,55,gonaives,19.4500,-72.6833,25.08,69,0,1.49,HT,1704633017
72,72,al quwayrah,29.8004,35.3116,25.00,31,0,1.26,JO,1704633030
80,80,sagua la grande,22.8067,-80.0756,21.55,89,0,1.77,CU,1704633039
203,203,haiku-pauwela,20.9219,-156.3051,21.13,88,0,2.06,US,1704633124
269,269,chauk,20.8833,94.8167,21.87,50,0,2.63,MM,1704633171
283,283,baud,20.8333,84.3167,22.65,41,0,0.96,IN,1704633177
287,287,necochea,-38.5473,-58.7368,23.57,58,0,1.55,AR,1704633179


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = null_values_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,port alfred,ZA,-33.5906,26.8910,77,
39,mar del plata,AR,-38.0023,-57.5575,77,
44,jamuria,IN,23.7000,87.0833,44,
55,gonaives,HT,19.4500,-72.6833,69,
72,al quwayrah,JO,29.8004,35.3116,31,
80,sagua la grande,CU,22.8067,-80.0756,89,
203,haiku-pauwela,US,20.9219,-156.3051,88,
269,chauk,MM,20.8833,94.8167,50,
283,baud,IN,20.8333,84.3167,41,
287,necochea,AR,-38.5473,-58.7368,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Setting parameters to search for a hotel
radius = 100000
params = { 
    'categories': 'accommodation.hotel',
    'limit': 20,
    'apiKey': geoapify_key,
}

# Printing a message to follow up the hotel search
print("Starting hotel search")

# Iterating through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # getting latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Adding filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port alfred - nearest hotel: No hotel found
mar del plata - nearest hotel: Gran hotel Mónaco
jamuria - nearest hotel: No hotel found
gonaives - nearest hotel: amiral kilik
al quwayrah - nearest hotel: Malakot of Wadi Rum
sagua la grande - nearest hotel: Hotel Telégrafo
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
chauk - nearest hotel: Royal Chauk
baud - nearest hotel: No hotel found
necochea - nearest hotel: Cabañas Río Quequén
abu tij - nearest hotel: أسيوط
champerico - nearest hotel: Hotel y Restaurante El Submarino
victorica - nearest hotel: Hotel Mazamet
nouadhibou - nearest hotel: Hotel valencia
la ligua - nearest hotel: Hotel Anchimallen
susangerd - nearest hotel: ۱۷ شهریور
guisa - nearest hotel: Hotel Sierra Maestra


,City,Country,Lat,Lng,Humidity,Hotel Name
20,port alfred,ZA,-33.5906,26.8910,77,No hotel found
39,mar del plata,AR,-38.0023,-57.5575,77,Gran hotel Mónaco
44,jamuria,IN,23.7000,87.0833,44,No hotel found
55,gonaives,HT,19.4500,-72.6833,69,amiral kilik
72,al quwayrah,JO,29.8004,35.3116,31,Malakot of Wadi Rum
80,sagua la grande,CU,22.8067,-80.0756,89,Hotel Telégrafo
203,haiku-pauwela,US,20.9219,-156.3051,88,Inn At Mama's Fish House
269,chauk,MM,20.8833,94.8167,50,Royal Chauk
283,baud,IN,20.8333,84.3167,41,No hotel found
287,necochea,AR,-38.5473,-58.7368,58,Cabañas Río Quequén


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot

# Converting the DataFrame to a GeoDataFrame
geometry = [Point(xy) for xy in zip(hotel_df['Lng'], hotel_df['Lat'])]
gdf = gpd.GeoDataFrame(hotel_df, geometry=geometry)

# Plotting the GeoDataFrame using hvplot
map_plot = gdf.hvplot.points('Lng', 'Lat', size='Humidity', geo=True, tiles='OSM',
                             color='City', hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'], scale = 0.9,
                             title='City Map with Humidity').opts(width=1000, height=600)

# Displaying the map 
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)